In [1]:
!pip install google-generativeai streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.7 MB/s eta 0:00:00


In [13]:
import google.generativeai as genai
import os
import json # For loading the JSON string into a Python dictionary

# Load the API key from Colab secrets
from google.colab import userdata
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API Key configured successfully!")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GOOGLE_API_KEY' not found. Please add it to Colab Secrets (key icon on the left).")
except Exception as e:
    print(f"An error occurred during API key configuration: {e}")

MODEL_NAME = "gemini-2.5-flash-preview-04-17" # If you have confirmed access

# For safety, let's try to instantiate the model to see if the API key and model name are okay
try:
    model_check = genai.GenerativeModel(MODEL_NAME)
    print(f"Successfully instantiated model: {MODEL_NAME}")
except Exception as e:
    print(f"ERROR: Could not instantiate model '{MODEL_NAME}'. Check API key and model availability. Error: {e}")

Gemini API Key configured successfully!
Successfully instantiated model: gemini-2.5-flash-preview-04-17


In [3]:
# --- 1. Our Master Prompt Template ---
# This is the prompt structure we perfected in AI Studio.
# We'll use f-strings in Python to insert the dynamic content.

MASTER_PROMPT_TEMPLATE = """You are "RegImpact AI", an AI legal tech assistant. Your purpose is to analyze a proposed new piece of legislation against provided snippets of existing law to identify potential impacts, overlaps, and conflicts.

Your response MUST be a single, valid JSON object. Do not include any text outside of this JSON object.

The JSON object must have the following top-level keys:
1. "proposed_legislation_summary": A brief (1-2 sentences) summary of the main purpose of the "Proposed New Legislation Text".
2. "existing_law_snippets_summary": A brief (1-2 sentences) summary of the main purpose of the "Provided Existing Law Snippets".
3. "impact_analysis": A list of objects. Each object in this list should describe a specific point of interaction, potential conflict, overlap, or significant change.
4. "key_impactful_phrases_in_new_law": A list of short phrases (3-7 words each) from the "Proposed New Legislation Text" that seem most significant in causing the identified impacts.

Each object within the "impact_analysis" list must have the following two keys:
1. "interaction_description": Your detailed description of a specific interaction, potential conflict, amendment, or overlap between the proposed legislation and the existing law snippets. Explain the nature of the impact.
2. "severity_rating_guess": Your estimated severity of this interaction/conflict. Must be one of these strings: "High", "Medium", "Low", or "Informational".
- "High": Indicates a direct conflict or a very significant change to existing obligations.
- "Medium": Indicates a notable overlap, a new requirement that builds upon existing law, or a potential ambiguity that needs clarification.
- "Low": Indicates a minor interaction or clarification with minimal change to substance.
- "Informational": Notes an overlap or connection that isn't a conflict but is relevant context.

Here is the information for your analysis:

**Proposed New Legislation Text:**
---
{new_legislation_text_placeholder}
---

**Provided Existing Law Snippets (Combined):**
---
{existing_law_snippets_placeholder}
---

Now, perform the analysis and provide the complete JSON output.
"""

# --- 2. Our Legislative Text Snippets ---
# The notebook will "know" where to look for these because we are defining them
# as Python variables right here. Later, our Streamlit app will allow the user
# to paste these in, but for testing our core function, we define them directly.

NEW_LEGISLATION_INPUT = """Algorithmic Pricing Transparency Mandate (Consumer AI Protection Initiative - Art. X)

(1) Where an undertaking employs an algorithmic system to dynamically determine the price of goods or services offered to consumers, the consumer must be clearly and conspicuously informed, prior to the presentation of the price, that the price has been algorithmically determined and is subject to dynamic adjustments. (2) Consumers must also be provided, upon simple request, with a concise explanation of the main parameters used by the algorithmic system to determine the price offered to them, excluding an undertaking's confidential business information that constitutes a trade secret. (3) This explanation shall include, at a minimum, the type of data that most significantly influenced the price presented.
"""

EXISTING_LAW_INPUT = """Existing Law Snippet 1 (from GDPR - Article 13(2)(f)):
The controller shall, at the time when personal data are obtained, provide the data subject with the following further information necessary to ensure fair and transparent processing: ... (f) the existence of automated decision-making, including profiling, referred to in Article 22(1) and (4) and, at least in those cases, meaningful information about the logic involved, as well as the significance and the envisaged consequences of such processing for the data subject.

Existing Law Snippet 2 (from GDPR - Article 22(1)):
The data subject shall have the right not to be subject to a decision based solely on automated processing, including profiling, which produces legal effects concerning him or her or similarly significantly affects him or her.

Existing Law Snippet 3 (from GDPR - Article 5(1)(a)):
Personal data shall be: (a) processed lawfully, fairly and in a transparent manner in relation to the data subject (‘lawfulness, fairness and transparency’);

Existing Law Snippet 4 (from Unfair Commercial Practices Directive - Article 7(1)):
A commercial practice shall be regarded as misleading if, in its factual context, taking account of all its features and circumstances and the limitations of the communication medium, it omits material information that the average consumer needs, according to the context, to take an informed transactional decision and thereby causes or is likely to cause the average consumer to take a transactional decision that he would not have taken otherwise.
"""

print("Master prompt template and legislative text snippets are defined.")

Master prompt template and legislative text snippets are defined.


In [14]:
def get_legislative_impact_analysis(new_legislation_text, existing_law_text):
    """
    Analyzes proposed legislation against existing law snippets using Gemini.

    Args:
        new_legislation_text (str): The text of the proposed new legislation.
        existing_law_text (str): Combined text of relevant existing law snippets.

    Returns:
        dict: A Python dictionary parsed from Gemini's JSON response,
              or None if an error occurs.
    """
    try:
        # 1. Construct the full prompt using our template and the provided texts
        full_prompt = MASTER_PROMPT_TEMPLATE.format(
            new_legislation_text_placeholder=new_legislation_text,
            existing_law_snippets_placeholder=existing_law_text
        )

        # 2. Instantiate the model (ensure MODEL_NAME is set from Cell 2)
        model = genai.GenerativeModel(MODEL_NAME)

        # 3. Generate content - critically asking for JSON output
        # The 'sdk-code-to-data.txt' used a 'contents' structure which is for chat history.
        # For a single turn, we can directly send the prompt.
        # We also need to explicitly ask for JSON using GenerationConfig.
        generation_config = genai.types.GenerationConfig(response_mime_type="application/json")

        print("Sending request to Gemini...")
        response = model.generate_content(
            full_prompt,
            generation_config=generation_config
        )
        print("Received response from Gemini.")

        # 4. Extract and parse the JSON
        # The Gemini API, when response_mime_type="application/json" is used,
        # should ideally put the parseable JSON directly in response.text.
        if response.text:
            # print(f"Raw response text: {response.text[:500]}...") # For debugging if needed
            return json.loads(response.text)
        else:
            # Fallback if .text is empty but parts might contain it (depends on exact model behavior)
            # This part might need adjustment based on the exact response structure for your model if .text is empty
            if response.candidates and response.candidates[0].content.parts:
                json_text = response.candidates[0].content.parts[0].text
                # print(f"Fallback raw response text from parts: {json_text[:500]}...") # For debugging
                return json.loads(json_text)
            else:
                print("ERROR: Gemini response was empty or not in the expected format.")
                print(f"Full response object: {response}") # Print the whole object for inspection
                return None

    except json.JSONDecodeError as e:
        print(f"ERROR: Could not decode JSON from Gemini response. Error: {e}")
        # It's useful to see the text that failed to parse
        if 'response' in locals() and hasattr(response, 'text'):
             print(f"Gemini response text that failed parsing: {response.text}")
        elif 'response' in locals() and response.candidates and response.candidates[0].content.parts:
             print(f"Gemini response text from parts that failed parsing: {response.candidates[0].content.parts[0].text}")
        return None
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during Gemini API call: {e}")
        # If possible, print the problematic response part
        if 'response' in locals():
            print(f"Problematic response object: {response}")
        return None

print("Analysis function 'get_legislative_impact_analysis' defined.")

Analysis function 'get_legislative_impact_analysis' defined.


In [15]:
# Let's test our function with the sample texts we defined in Cell 3
print("Attempting to analyze legislative texts...")
analysis_result = get_legislative_impact_analysis(NEW_LEGISLATION_INPUT, EXISTING_LAW_INPUT)

if analysis_result:
    print("\n--- Successfully retrieved and parsed analysis! ---")
    # Print a summary or part of the result to verify
    print(f"\nProposed Legislation Summary from AI: {analysis_result.get('proposed_legislation_summary')}")
    print(f"\nNumber of impact points identified: {len(analysis_result.get('impact_analysis', []))}")

    # You can pretty-print the whole JSON to inspect it:
    # import pprint
    # print("\nFull analysis result:")
    # pprint.pprint(analysis_result)
else:
    print("\n--- Analysis failed. Check error messages above. ---")

Attempting to analyze legislative texts...
Sending request to Gemini...
Received response from Gemini.

--- Successfully retrieved and parsed analysis! ---

Proposed Legislation Summary from AI: This proposal requires businesses using algorithmic pricing for consumers to disclose that the price is algorithmically determined and dynamic before presenting it, and to provide a concise explanation of the main parameters and data used upon consumer request.

Number of impact points identified: 3


In [16]:
%%writefile app.py
# This magic command tells Colab to write the contents of this cell to a file named "app.py"

import streamlit as st
import google.generativeai as genai
import os
import json

# --- Configuration and Helper Functions ---

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY_STREAMLIT")

configured_successfully = False
if GOOGLE_API_KEY:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        configured_successfully = True
    except Exception as e:
        print(f"Streamlit App (startup): Error configuring Gemini API: {e}")
else:
    print("Streamlit App (startup): CRITICAL - GOOGLE_API_KEY_STREAMLIT environment variable not found.")

APP_MODEL_NAME = "gemini-2.5-flash-preview-04-17" # Using your specified model

MASTER_PROMPT_TEMPLATE = """You are "RegImpact AI", an AI legal tech assistant. Your purpose is to analyze a proposed new piece of legislation against provided snippets of existing law to identify potential impacts, overlaps, and conflicts.

Your response MUST be a single, valid JSON object. Do not include any text outside of this JSON object. All keys and string values within the JSON object MUST be enclosed in double quotes. Ensure there are no trailing commas.

The JSON object must have the following top-level keys:
1. "proposed_legislation_summary": A brief (1-2 sentences) summary of the main purpose of the "Proposed New Legislation Text".
2. "existing_law_snippets_summary": A brief (1-2 sentences) summary of the main purpose of the "Provided Existing Law Snippets".
3. "impact_analysis": A list of objects. Each object in this list should describe a specific point of interaction, potential conflict, overlap, or significant change.
4. "key_impactful_phrases_in_new_law": A list of short phrases (3-7 words each) from the "Proposed New Legislation Text" that seem most significant in causing the identified impacts.

Each object within the "impact_analysis" list must have the following two keys:
1. "interaction_description": Your detailed description of a specific interaction, potential conflict, amendment, or overlap between the proposed legislation and the existing law snippets. Explain the nature of the impact. All internal quotes within this description must be properly escaped for JSON.
2. "severity_rating_guess": Your estimated severity of this interaction/conflict. Must be one of these strings: "High", "Medium", "Low", or "Informational".
- "High": Indicates a direct conflict or a very significant change to existing obligations.
- "Medium": Indicates a notable overlap, a new requirement that builds upon existing law, or a potential ambiguity that needs clarification.
- "Low": Indicates a minor interaction or clarification with minimal change to substance.
- "Informational": Notes an overlap or connection that isn't a conflict but is relevant context.

Here is the information for your analysis:

**Proposed New Legislation Text:**
---
{new_legislation_text_placeholder}
---

**Provided Existing Law Snippets (Combined):**
---
{existing_law_snippets_placeholder}
---

Now, perform the analysis and provide the complete JSON output. Ensure strict JSON formatting.
""" # Added more specific JSON formatting instructions

# Default texts for the UI text areas ( 그대로 유지 )
DEFAULT_NEW_LAW_TEXT = """Algorithmic Pricing Transparency Mandate (Consumer AI Protection Initiative - Art. X)

(1) Where an undertaking employs an algorithmic system to dynamically determine the price of goods or services offered to consumers, the consumer must be clearly and conspicuously informed, prior to the presentation of the price, that the price has been algorithmically determined and is subject to dynamic adjustments. (2) Consumers must also be provided, upon simple request, with a concise explanation of the main parameters used by the algorithmic system to determine the price offered to them, excluding an undertaking's confidential business information that constitutes a trade secret. (3) This explanation shall include, at a minimum, the type of data that most significantly influenced the price presented.
"""

DEFAULT_EXISTING_LAW_TEXT = """Existing Law Snippet 1 (from GDPR - Article 13(2)(f)):
The controller shall, at the time when personal data are obtained, provide the data subject with the following further information necessary to ensure fair and transparent processing: ... (f) the existence of automated decision-making, including profiling, referred to in Article 22(1) and (4) and, at least in those cases, meaningful information about the logic involved, as well as the significance and the envisaged consequences of such processing for the data subject.

Existing Law Snippet 2 (from GDPR - Article 22(1)):
The data subject shall have the right not to be subject to a decision based solely on automated processing, including profiling, which produces legal effects concerning him or her or similarly significantly affects him or her.

Existing Law Snippet 3 (from GDPR - Article 5(1)(a)):
Personal data shall be: (a) processed lawfully, fairly and in a transparent manner in relation to the data subject (‘lawfulness, fairness and transparency’);

Existing Law Snippet 4 (from Unfair Commercial Practices Directive - Article 7(1)):
A commercial practice shall be regarded as misleading if, in its factual context, taking account of all its features and circumstances and the limitations of the communication medium, it omits material information that the average consumer needs, according to the context, to take an informed transactional decision and thereby causes or is likely to cause the average consumer to take a transactional decision that he would not have taken otherwise.
"""


@st.cache_data(show_spinner=False)
def get_legislative_impact_analysis_streamlit(new_legislation_text, existing_law_text, model_to_use):
    global configured_successfully

    if not configured_successfully:
         st.error("API Key not configured correctly at app startup. Cannot make analysis. Check console logs from Streamlit startup.")
         return None
    if not GOOGLE_API_KEY:
         st.error("API Key (GOOGLE_API_KEY_STREAMLIT) became unavailable. Ensure Cell 7 in Colab was run.")
         return None

    returned_text_for_debugging = "" # Initialize variable to store text from Gemini
    try:
        full_prompt = MASTER_PROMPT_TEMPLATE.format(
            new_legislation_text_placeholder=new_legislation_text,
            existing_law_snippets_placeholder=existing_law_text
        )

        model = genai.GenerativeModel(model_to_use)
        generation_config = genai.types.GenerationConfig(response_mime_type="application/json")

        with st.spinner(f"RegImpact AI is analyzing using {model_to_use}... please wait..."):
            response = model.generate_content(
                full_prompt,
                generation_config=generation_config
            )

        # Store the raw text before attempting to parse
        if response.text:
            returned_text_for_debugging = response.text
            return json.loads(returned_text_for_debugging)
        else: # Fallback
            if response.candidates and response.candidates[0].content.parts:
                if hasattr(response.candidates[0].content.parts[0], 'text'):
                    returned_text_for_debugging = response.candidates[0].content.parts[0].text
                    return json.loads(returned_text_for_debugging)
                else:
                    st.error("ERROR: Gemini response part did not contain text.")
                    return None
            else:
                st.error("ERROR: Gemini response was empty or not in the expected format.")
                return None

    except json.JSONDecodeError as e:
        st.error(f"ERROR: Could not decode JSON from Gemini. Error: {e}")
        # Print the problematic text to the Colab console where Streamlit logs
        print("--- DEBUG: Problematic text from Gemini that failed JSON parsing ---")
        print(returned_text_for_debugging)
        print("--- END DEBUG ---")
        return None
    except Exception as e:
        st.error(f"ERROR: An unexpected error occurred during analysis: {e}")
        # print(f"DEBUG: Problematic response object: {response if 'response' in locals() else 'Response object not available'}")
        return None

# --- Streamlit UI (remains the same as your last fully working UI version) ---
st.set_page_config(page_title="RegImpact AI Analyzer", layout="wide")

st.sidebar.title("About RegInSight")
st.sidebar.info(
    f"""
    RegInSight uses Google's Gemini AI model **({APP_MODEL_NAME})**
    to perform a comparative textual analysis between proposed legislation
    (or guidelines) and existing legal snippets.
    It aims to highlight potential areas of interaction, conflict, or
    overlap to assist legal and policy professionals in their detailed review process.
    \n\nA Hackathon Project for the Legal Frontier Hackathon: AI & Beyond
    """
)
st.sidebar.warning(
    """
    **Disclaimer:** This AI-generated analysis is for informational and illustrative
    purposes only and **does not constitute legal advice.** It provides a
    preliminary assessment based on the provided texts. Always consult with
    qualified legal professionals for any legal matters.
    """
)

st.title("RegImpact AI ⚖️ Analyzer")
st.markdown("Analyze proposed legislation against existing laws to identify potential impacts.")

if not configured_successfully: # Display prominent error if API key setup failed
    st.error("Streamlit App: CRITICAL - Gemini API Key was not configured successfully on startup. Analysis will fail. Please check the Colab console output when Streamlit started (after running Cell 8) for error messages regarding API key configuration. Ensure the GOOGLE_API_KEY_STREAMLIT environment variable was correctly set by Cell 7 before running Cell 8.")

col1, col2 = st.columns(2)

with col1:
    st.subheader("Proposed New Legislation Text")
    new_law_input = st.text_area("Paste the new proposed legislation text here:",
                                 value=DEFAULT_NEW_LAW_TEXT,
                                 height=350,
                                 key="new_law_text_area")

with col2:
    st.subheader("Relevant Existing Law Snippets")
    existing_law_input = st.text_area("Paste relevant snippets from existing laws here (combine multiple snippets if needed):",
                                      value=DEFAULT_EXISTING_LAW_TEXT,
                                      height=350,
                                      key="existing_law_text_area")

if st.button("Analyze Impact with RegInSight AI", type="primary", use_container_width=True, key="analyze_button"):
    if not configured_successfully:
        st.error("Critical Error: API Key is not configured for the Streamlit app. Analysis cannot proceed.")
    elif new_law_input and existing_law_input:
        analysis_result = get_legislative_impact_analysis_streamlit(new_law_input, existing_law_input, APP_MODEL_NAME)

        if analysis_result:
            st.markdown("---")
            st.subheader("Analysis Report")

            summary_prop = analysis_result.get("proposed_legislation_summary", "Not available.")
            summary_exist = analysis_result.get("existing_law_snippets_summary", "Not available.")
            key_phrases = analysis_result.get("key_impactful_phrases_in_new_law", [])
            impact_points = analysis_result.get("impact_analysis", [])

            st.markdown(f"**Summary of Proposed Legislation:** {summary_prop}")
            st.markdown(f"**Summary of Existing Law Snippets:** {summary_exist}")

            if key_phrases:
                st.markdown("---")
                st.markdown("**Key Impactful Phrases in New Law:**")
                for phrase in key_phrases:
                    st.markdown(f"- `{phrase}`")

            if impact_points:
                st.markdown("---")
                st.markdown("### Detailed Impact Analysis:")
                for i, point in enumerate(impact_points):
                    desc = point.get("interaction_description", "No description.")
                    severity = point.get("severity_rating_guess", "N/A").upper()
                    expander_title_desc = desc[:70] + "..." if len(desc) > 70 else desc

                    with st.expander(f"Impact Point {i+1}: ({severity}) - {expander_title_desc}", expanded= i==0):
                        st.markdown(f"**Interaction Description:**")
                        st.write(desc)
                        st.markdown(f"**Severity Rating Guess:** {severity}")
            elif isinstance(impact_points, list) and not impact_points:
                 st.info("The AI analysis did not identify specific impact points based on the provided texts.")
            else:
                st.warning("No specific impact points were identified in the analysis or the analysis format was unexpected.")
    else:
        st.warning("Please provide text for both proposed and existing legislation.")

st.markdown("---")
st.info(
    """
    **Guidance for Human Review:**
    This AI analysis provides a preliminary impact assessment. It is **essential** for qualified legal and policy experts to conduct a thorough human review of these AI-identified points and the full legislative texts before making any decisions or drawing definitive conclusions. AI is a tool to augment, not replace, expert human judgment.
    """
)
st.caption(f"RegInSight - AI Legislative Impact Analyzer (Demo Model: {APP_MODEL_NAME})")

Overwriting app.py


In [17]:
import os
from google.colab import userdata
# Get the key from Colab secrets
colab_api_key = userdata.get('GOOGLE_API_KEY')

# Set it as an environment variable that app.py will look for
# We use a slightly different name to be sure which one is being picked up
os.environ['GOOGLE_API_KEY_STREAMLIT'] = colab_api_key

print("GOOGLE_API_KEY_STREAMLIT environment variable has been set for this Colab session.")
print("(It will be cleared when this Colab runtime restarts.)")

GOOGLE_API_KEY_STREAMLIT environment variable has been set for this Colab session.
(It will be cleared when this Colab runtime restarts.)


In [18]:
from pyngrok import ngrok, conf
from google.colab import userdata # To get secrets
import os

# Terminate any existing tunnels if you re-run this cell
print("Killing any existing ngrok tunnels...")
ngrok.kill()

# --- Configure ngrok Authtoken ---
NGROK_AUTHTOKEN = ""
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN') # Use the exact name you gave the secret
    print("Successfully retrieved NGROK_AUTHTOKEN from Colab Secrets.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'NGROK_AUTHTOKEN' not found in Colab Secrets.")
    print("Please add your ngrok authtoken to Colab Secrets (key icon on the left).")
except Exception as e:
    print(f"An error occurred retrieving the ngrok authtoken: {e}")

if NGROK_AUTHTOKEN:
    try:
        # conf.get_default().auth_token = NGROK_AUTHTOKEN # Old way
        # The new way is often to write it to the ngrok config file path pyngrok will use
        # Or to set it before connecting via ngrok.set_auth_token()
        print(f"Configuring ngrok with authtoken...")
        ngrok.set_auth_token(NGROK_AUTHTOKEN) # Preferred way to set token for pyngrok
        print("Ngrok authtoken configured.")
    except Exception as e:
        print(f"ERROR: Failed to configure ngrok with authtoken: {e}")
        # Fallback: Try writing to the default config file path if direct set fails
        # This is more of a workaround and might not always be reliable in Colab's environment
        try:
            print("Attempting fallback ngrok configuration...")
            ngrok_config_path = os.path.join(os.path.expanduser("~"), ".ngrok2", "ngrok.yml")
            os.makedirs(os.path.dirname(ngrok_config_path), exist_ok=True)
            with open(ngrok_config_path, "w") as f:
                f.write(f"authtoken: {NGROK_AUTHTOKEN}\nversion: 2")
            print(f"Ngrok authtoken written to {ngrok_config_path}. Please re-run this cell if connection still fails.")
        except Exception as e_fallback:
            print(f"Fallback ngrok configuration also failed: {e_fallback}")

else:
    print("Ngrok authtoken not available. Tunneling will likely fail.")


# Run streamlit in the background. 8501 is the default streamlit port.
# The output is redirected to /dev/null to keep the Colab cell clean.
# The '&' at the end runs it in the background.
print("Starting Streamlit app in the background...")
!streamlit run app.py --server.port 8501 &>/dev/null&

# Open a tunnel to the streamlit port 8501
if NGROK_AUTHTOKEN: # Only attempt to connect if we have a token
    try:
        print("Connecting ngrok tunnel...")
        public_url = ngrok.connect(8501) # pyngrok should now use the configured token
        print(f"Your Streamlit app is live at: {public_url}")
        print("If the app doesn't load, check the output above for any errors during Streamlit startup or ngrok connection.")
    except Exception as e:
        print(f"ERROR starting ngrok tunnel: {e}")
        print("Please ensure your NGROK_AUTHTOKEN is correct and valid.")
        print("You might need to restart the Colab runtime after adding/updating the secret if issues persist.")
else:
    print("Cannot start ngrok tunnel because authtoken is missing.")

Killing any existing ngrok tunnels...
Successfully retrieved NGROK_AUTHTOKEN from Colab Secrets.
Configuring ngrok with authtoken...
Ngrok authtoken configured.
Starting Streamlit app in the background...
Connecting ngrok tunnel...
Your Streamlit app is live at: NgrokTunnel: "https://5acb-35-237-172-71.ngrok-free.app" -> "http://localhost:8501"
If the app doesn't load, check the output above for any errors during Streamlit startup or ngrok connection.


In [19]:
import textwrap

# --- Project Information (Customize as needed) ---
project_title = "🚀 RegInSight: AI-Powered Legislative Impact Forecaster"
hackathon_name = "Legal Frontier Hackathon: AI & Beyond"
team_name = "[PROJECTO 4 PATAS / Jan Pavuk]" # <<<<<<<<<<<<<<<  ACTION: Fill this in!
current_date = "May 17-18, 2025"
APP_MODEL_NAME = "gemini-2.5-flash-preview-04-17"

# --- Core Logic of the App (Concise Bullet Points) ---
what_it_does_concise = [
    "**Analyzes New and Proposed Laws vs. Existing Laws:** Identifies potential conflicts, overlaps, & impacts.",
    "**AI-Powered Summaries & Insights:** Highlights key changes & impactful phrases using Gemini AI.",
    "**Accelerates Initial Review:** Significantly speeds up the work for legal & policy professionals."
]

how_it_works_concise = [
    "**Simple Text Input:** Users paste proposed legislation & relevant existing legal snippets.",
    f"**Gemini AI Core:** `{APP_MODEL_NAME}` performs comparative textual analysis via a sophisticated prompt.",
    "**Structured JSON to Web UI:** Delivers clear, actionable insights through a Streamlit interface."
]

target_users_concise = [
    "**Government & Legislatures:** Legislative drafters, policy analysts.",
    "**Corporate Legal Departments:** In-house counsel in regulated industries.",
    "**Law Firms & Consultants:** Specialists in regulatory compliance & public policy."
]

# --- Market & Value (Concise) ---
market_value_concise = [
    "**Vast Market:** Hundreds of new EU laws/regulations annually requiring analysis by thousands of entities.",
    "**Significant Cost Savings:** Estimates show potential savings of **~€900 per proposal reviewed** by reducing manual effort by **~30% (6+ hours)**.",
    "**Strategic Advantage:** Frees up experts for high-value analysis, not just manual cross-referencing."
]

# --- Next Steps (Concise) ---
next_steps_concise = [
    "**Pilot Programs:** Engage target users for real-world feedback & validation.",
    "**Legal Database Integration:** Connect with actual legal repositories for broader scope.",
    "**Enhanced AI Features:** Develop deeper contextual understanding & knowledge graph capabilities."
]

# --- Construct Markdown Output ---
markdown_output = f"""
# {project_title}
### *By: {team_name} | {hackathon_name} | {current_date}*

---

## 🎯 What RegInSight Does:
"""
for item in what_it_does_concise:
    markdown_output += f"- {item}\n"

markdown_output += """
---

## ⚙️ How It Works:
"""
for item in how_it_works_concise:
    markdown_output += f"- {item}\n"

markdown_output += """
---

## 👥 Target Users:
"""
for item in target_users_concise:
    markdown_output += f"- {item}\n"

markdown_output += """
---

## 📈 Market Opportunity & Value Proposition:
"""
# No subheadings here for conciseness on one slide, integrate into points
for item in market_value_concise:
    markdown_output += f"- {item}\n"

markdown_output += """
---

## 🚀 Next Steps & Future Vision:
"""
for item in next_steps_concise:
    markdown_output += f"- {item}\n"

markdown_output += """
---
### *RegInSight: Augmenting Legal Expertise with AI for a Clearer Legislative Future.*
"""

# --- Print the Markdown ---
# print(markdown_output) # You can uncomment this if you want to see the raw markdown for copying

# --- Display Formatted Markdown in Colab ---
from IPython.display import display, Markdown
display(Markdown(markdown_output))


# 🚀 RegInSight: AI-Powered Legislative Impact Forecaster
### *By: [PROJECTO 4 PATAS / Jan Pavuk] | Legal Frontier Hackathon: AI & Beyond | May 17-18, 2025*

---

## 🎯 What RegInSight Does:
- **Analyzes New and Proposed Laws vs. Existing Laws:** Identifies potential conflicts, overlaps, & impacts.
- **AI-Powered Summaries & Insights:** Highlights key changes & impactful phrases using Gemini AI.
- **Accelerates Initial Review:** Significantly speeds up the work for legal & policy professionals.

---

## ⚙️ How It Works:
- **Simple Text Input:** Users paste proposed legislation & relevant existing legal snippets.
- **Gemini AI Core:** `gemini-2.5-flash-preview-04-17` performs comparative textual analysis via a sophisticated prompt.
- **Structured JSON to Web UI:** Delivers clear, actionable insights through a Streamlit interface.

---

## 👥 Target Users:
- **Government & Legislatures:** Legislative drafters, policy analysts.
- **Corporate Legal Departments:** In-house counsel in regulated industries.
- **Law Firms & Consultants:** Specialists in regulatory compliance & public policy.

---

## 📈 Market Opportunity & Value Proposition:
- **Vast Market:** Hundreds of new EU laws/regulations annually requiring analysis by thousands of entities.
- **Significant Cost Savings:** Estimates show potential savings of **~€900 per proposal reviewed** by reducing manual effort by **~30% (6+ hours)**.
- **Strategic Advantage:** Frees up experts for high-value analysis, not just manual cross-referencing.

---

## 🚀 Next Steps & Future Vision:
- **Pilot Programs:** Engage target users for real-world feedback & validation.
- **Legal Database Integration:** Connect with actual legal repositories for broader scope.
- **Enhanced AI Features:** Develop deeper contextual understanding & knowledge graph capabilities.

---
### *RegInSight: Augmenting Legal Expertise with AI for a Clearer Legislative Future.*


In [10]:
# Generate README.md content
readme_content = f"""
# {project_title}
## {hackathon_name} | Team: {team_name} | Date: {current_date}

---

## Project Overview

RegInSight is an AI-powered tool designed to streamline the process of analyzing new or proposed legislation against existing legal frameworks. By leveraging the capabilities of Google's Gemini AI, it identifies potential impacts, conflicts, and overlaps, significantly accelerating the initial review phase for legal and policy professionals.

---

## What it Does

*   **Analyzes New vs. Existing Laws:** Automatically compares proposed legal text with provided snippets of existing law.
*   **Identifies Impacts:** Highlights potential conflicts, overlaps, and significant changes introduced by the new legislation.
*   **Generates AI Summaries:** Provides concise summaries of both the proposed legislation and the existing law snippets.
*   **Extracts Key Phrases:** Identifies the most impactful phrases within the new legislation that are likely to cause interaction with existing laws.
*   **Provides Severity Ratings:** Offers an estimated severity rating for each identified interaction (High, Medium, Low, Informational).

---

## How it Works

1.  **Input:** Users paste the text of the proposed new legislation and relevant existing law snippets into the web interface.
2.  **AI Analysis:** The application sends the input texts to the configured Gemini AI model (`{APP_MODEL_NAME}`).
3.  **Prompt Engineering:** A carefully crafted prompt guides the AI to perform a comparative textual analysis and structure its output as a JSON object.
4.  **JSON Output:** The AI returns a JSON object containing summaries, key phrases, and a detailed list of identified impacts with severity ratings.
5.  **Web UI:** A Streamlit web application parses the JSON output and presents the analysis in a clear, organized, and interactive format.

---

## Target Users

*   **Government & Legislatures:** Legislative drafters and policy analysts.
*   **Corporate Legal Departments:** In-house counsel responsible for compliance in regulated industries.
*   **Law Firms & Consultants:** Specialists in regulatory compliance, public policy, and legislative analysis.

---

## Market Opportunity & Value Proposition

The complexity and volume of new legislation, particularly in areas like technology and AI, create a significant need for efficient analysis tools. RegInSight offers:

*   **Vast Market:** Applies to the analysis of hundreds of new laws and regulations annually across numerous jurisdictions.
*   **Significant Cost Savings:** Reduces the manual effort required for initial cross-referencing, potentially saving hours per analysis.
*   **Strategic Advantage:** Allows legal professionals to focus on higher-value strategic analysis rather than time-consuming manual comparison.

---

## Next Steps & Future Vision

*   **Pilot Programs:** Collaborate with target users to gather real-world feedback and refine the tool.
*   **Legal Database Integration:** Explore connecting RegInSight to legal databases for automated retrieval of relevant existing laws.
*   **Enhanced AI Capabilities:** Investigate further development of AI features for deeper contextual understanding, knowledge graph creation, and more nuanced analysis.

---

## Disclaimer

This AI-generated analysis is for informational and illustrative purposes only and **does not constitute legal advice.** It provides a preliminary assessment based on the provided texts. Always consult with qualified legal professionals for any legal matters.

---

### *RegInSight: Augmenting Legal Expertise with AI for a Clearer Legislative Future.*
"""

# Write the content to a README.md file
with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md generated successfully!")

README.md generated successfully!
